In [2]:
# from flask_other.app import app
from flask import Flask
import os,sys
sys.path.append('../')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests

##################################
url = 'http://localhost:8889/chatbotv1'


def get_data(sentence):
    
    response = requests.get(url, timeout=10, params={'data':sentence, 
                                                     'action':'chat', 
                                                     'sessionId':cache.sessionId})
    
    if response.status_code == 200:
        req = response.json()
        if req.get('status').lower() == 'successful':
            msg = req.get('message')
        else:
            msg = req

        return msg
    else:
        return 'Internet Connection Issue, error code {}'.format(response.status_code)



@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )
@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            query = request.form.get('query')#前端查询的内容
            if query:

                    print('query',query)
                    # time.sleep(5)
                    result = get_data(query)
                    print('result',result)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('query duplicated! Use same id to save')
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                return str('MM出故障啦')


    else:
        return 'ok'
    
    
@app.route("/newConversation", methods= ["POST"])
def new_conversation():
    cache.new_conversation()
    print('new conversation called!!!!!!!!!!!!!!!!!!!!!!!')

class Cache:
    def __init__(self):
        self.new_conversation()
            
    def new_conversation(self):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                self.sessionId = msg['message']['sessionId']
                print(self.sessionId)
            else:
                print(req.content)
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.content)


        
cache = Cache()   




chatbotdb
0


In [3]:
app.debug = False
port = 6006
print('running at http://10.0.24.31:{}'.format(port))
app.run(host='0.0.0.0',port=port)    #这样用来监听所有的ip，团队调试用

 * Running on http://0.0.0.0:6006/ (Press CTRL+C to quit)


running at http://10.0.24.31:6006


172.18.0.1 - - [16/Jun/2018 19:04:52] "GET / HTTP/1.1" 200 -
172.18.0.1 - - [16/Jun/2018 19:04:55] "POST /dataFromAjax_post HTTP/1.1" 404 -
[2018-06-16 19:05:01,243] ERROR in app: Exception on /newConversation [POST]
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1615, in full_dispatch_request
    return self.finalize_request(rv)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1630, in finalize_request
    response = self.make_response(rv)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1725, in make_response
    raise ValueError('View function did not return a response')
ValueError: View function did not return a response
172.18.0.1 - - [16/Jun/2018 19:05:01] "POST /newConversation HTTP/1.1" 500 -
172.18.0.1 - - [16/Jun/2018 19:05:

1
new conversation called!!!!!!!!!!!!!!!!!!!!!!!
query d
result Internet Connection Issue, error code 500
saving to mongo db successfully!


172.18.0.1 - - [16/Jun/2018 19:05:13] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [16/Jun/2018 19:05:13] "POST /predict HTTP/1.1" 200 -


...

query d
result 你好，这里是H催收公司，请问是罗巍先生吗？
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "8277e0910d750195b448797616e091ad" }
query duplicated! Use same id to save


172.18.0.1 - - [16/Jun/2018 19:05:59] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [16/Jun/2018 19:05:59] "POST /predict HTTP/1.1" 200 -


query m
result 你好，我是H催收公司的客服小催，您之前借贷了H公司5万块钱，约定在2018年5月1日还清。您已还款3万2千块钱，但因逾期未还产生了相应的利息和延迟还款费用，现在一共需要还2万块钱，其中1.2万本金，1500利息，500延迟还款手续费用，请问您打算什么时候处理下呢？
saving to mongo db successfully!


172.18.0.1 - - [16/Jun/2018 19:06:00] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [16/Jun/2018 19:06:00] "POST /predict HTTP/1.1" 200 -


query d
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
